# Hyperion Hub Code Generator
## Oracle Developer
c. marxvim 2024

https://marxvim-1.gitbook.io/oracle-dev

In [2]:
%%capture --no-stderr

UsageError: %%capture is a cell magic, but the cell body is empty.


In [3]:
import openai
import re
from github import Github
from github import Auth
import os
from dotenv import load_dotenv
import autogen

load_dotenv()

open_ai_key = os.getenv("OPEN_AI_KEY")
git_token = Auth.Token(os.getenv("GITHUB_KEY"))

base_urls = None  # You can specify API base URLs if needed. eg: localhost:8000
api_type = "openai"  # Type of API, e.g., "openai" or "aoai".
api_version = None  # Specify API version if needed.

config_list = [
    {
        'model': 'gpt-4',
        'api_key': open_ai_key,
    }
]

config_list_gpt4 = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4", "gpt-4-0314", "gpt4", "gpt-4-32k", "gpt-4-32k-0314", "gpt-4-32k-v0314"],
    },
)



# TODO LIST
1. Use LangChain to connect to Anthropic, OpenAI, Azure models
2. Use Chroma or Pinecode to store Vector Database Data from Github readme and ther `relevant user provided files`
3. Use LangChain & OpenAI to generate Embeddings
4. Use LangChain to implement RAG properly

In [4]:
def get_repos(auth_token=None):
    auth = auth_token
    github_repo = Github(auth=auth)
    git_user = github_repo.get_user()
    return git_user.get_repos()

In [5]:
def prompt_gpt(prompt=""):
    openai.api_key = open_ai_key
    response = openai.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

def extract_mermaid_script(gpt_response):
    # Regex pattern to find content within ```mermaid ... ```
    pattern = r"```mermaid([\s\S]*?)```"
    match = re.search(pattern, gpt_response)
    if match:
        return match.group(1).strip()  # Returns only the script part
    else:
        return "none"  # If no script is found
    
def extract_readme(user_repos=None):
    # for repo in user_repos:
    #     print(repo.name)

    aRepo = user_repos[10]
    readme = aRepo.get_readme()
    if readme:
        # Print the README content
        readme_content = readme.decoded_content.decode('utf-8')
        return readme_content
    else:
        return None

In [6]:
user_prompt = "create a python function that implements a binary search algorithm"

mermaid_script_prompt = (
    "Using Mermaid diagram syntax, create a script for the following task: "
    + user_prompt
    + ". Please provide only the Mermaid script without any additional explanation, comments, or words. If a script cannot be generated for this task, simply return 'none'."
)


generated_mermaid_script = prompt_gpt(mermaid_script_prompt)
mermaid_script = extract_mermaid_script(generated_mermaid_script)

user_repos = get_repos(git_token)
readme = extract_readme(user_repos)
#print(readme)
print(mermaid_script)

none


In [7]:
llm_config = {"config_list": config_list, "cache_seed": 42}
user_proxy = autogen.UserProxyAgent(
   name="User_proxy",
   system_message="A human admin.",
   code_execution_config={"last_n_messages": 2, "work_dir": "groupchat"},
   human_input_mode="NEVER"
)
coder = autogen.AssistantAgent(
    name="Coder",
    llm_config=llm_config,
)
pm = autogen.AssistantAgent(
    name="Product_manager",
    system_message="Creative in software product ideas.",
    llm_config=llm_config,
)

qat = autogen.AssistantAgent(
    name="Quality_assurance_tester",
    system_message="primarily focused on verifying that the software meets the specified quality standards, adheres to coding standards, and is free from defects or bugs",
    llm_config=llm_config,
)
groupchat = autogen.GroupChat(agents=[user_proxy, coder, qat], messages=[], max_round=12)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

user_proxy.initiate_chat(manager, message=user_prompt)


User_proxy (to chat_manager):

We need a queue system that holds the IDs and names of incoming patients in a hospital. The function needs to be created using python

--------------------------------------------------------------------------------


Coder (to chat_manager):

Sure, we can model this scenario with a simple queue using a list in Python. We will define two functions: one function to add a patient to the queue and another function to remove a patient from the queue. When a patient is added, they are added to the end of the queue. When a patient is removed, the patient at the front of the queue is removed as per the First-In-First-Out (FIFO) property of a queue. 

Let's start by creating an empty list named `queue` to store our patients. A patient can be represented as a dictionary with keys `ID` and `name`.

Here is the Python code that implements this model:

```python
queue = []

def add_patient(ID, name):
    patient = {"ID": ID, "name": name}
    queue.append(patient)
    print(f"Patient {name} with ID {ID} has been added to the queue.")

def remove_patient():
    if len(queue) == 0:
        print("No patients in the queue.")
    else:
        patient = queue.pop(0)
        print(f"Patient {patient['name']} with ID

User_proxy (to chat_manager):

exitcode: 0 (execution succeeded)
Code output: 
Patient John Doe with ID 101 has been added to the queue.
Patient Jane Doe with ID 102 has been added to the queue.
Patient John Doe with ID 101 has been removed from the queue.


--------------------------------------------------------------------------------
Quality_assurance_tester (to chat_manager):

It seems that the code is working as expected. It successfully added two patients, "John Doe" and "Jane Doe", to the queue. Then, it removed the first patient "John Doe" from the queue, demonstrating the First-In-First-Out (FIFO) behavior of a queue. 

Is there anything else you want to add or any other functionality you want to implement?


--------------------------------------------------------------------------------
User_proxy (to chat_manager):



--------------------------------------------------------------------------------
Coder (to chat_manager):

TERMINATE

---------------------------------------